# Lista de aristas doblemente conectadas
## Librerias

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Clases Iniciales

### Clase Vertice

In [3]:
class vertex:
    def __init__(self, x , y, name):
        self.x = x
        self.y = y
        self.name = name
        self.edge_inci = None
        
    #Agregar arista incidente
    def add_edge_inci(self,e):
        self.edge_inci = e

    #Relacion de orden
    def r_orden(self,q):
        if self.y > q.y:
            point = self
        elif self.y == q.y:
            if self.x < q.x:
                point = self
            else:
                point = q
        else:
            point = q
        return point
    

### Clase Semi-Arista

In [4]:
class half_edge:
    def __init__(self,start,end,name):
        self.origin = start #Origen
        self.queue = end #final
        self.name = name #nombre
        self.next = None #Siguiente
        self.prev = None #Previo
        self.face = None #Cara incidente
        self.twin = twin(self) # Gemelo

    global twin
    def twin(self): #Gemelo
        return self.queue, self.origin 

    def add_next(self,n):
        self.next = n #Siguiente
    
    def add_prev(self,pv):
        self.prev = pv #Previo

    def add_face(self,f):
        self.face = f #Cara incidente    


### Clase Cara

In [5]:
class face:
    def __init__(self,n):
        self.name = n #Nombre
        self.exterior = None #frontera exterior
        self.interior = None #frontera interior

## Para tener en cuenta

In [ ]:
# P0 : Punto arista origen
# P1 : Punto arista next origin
# P2 : Punto arista next cola
def Producto_Cruz(P0,P1,P2):
    d = ((P1.x-P0.x)*(P2.y-P0.y))-((P2.x-P0.x)*(P1.y-P0.y))
    return d

In [ ]:
# P0 : Punto arista origen
# P1 : Punto arista next origin
# P2 : Punto arista next cola
def orientacion(P0,P1,P2):
    d = Producto_Cruz(P0,P1,P2)
    if d>0:
        a = True
        print("P2 se hace un giro en sentido antihoraro de P1")
    else:
        a = False
        print("P2 esta girando en sentido horario de P1")
    return a 

### Clase DCEL

In [53]:
class DCEL():
    # Ingresaremos unicamente la lista de objetos vertices
    def __init__(self,vertex):
        self.list_edge = []
        self.list_vertex = vertex
        self.list_face = []

    # A partir de los vertices vamos a generar la lista de caras y semi-aritas
    def create_edge_face(self):
        self.create_half_edge()

    #Generar los gemelos con sus caracteristicas
    def _add_twin(self):
        twins = []
        twins_sorted = []
        edges_copy = self.list_edge.copy()
        for i in range(len(edges_copy)):
            t_start, t_end = twin(edges_copy[i])
            twins.append(half_edge(t_start,t_end,'te'+str(i+1)))

        print('Entro2')
        
        # Organizamos para que quede de manera ciclica y agremaos el next
        origen = twins[0].queue
        a = True
        while(a != False):
            edge_next = [i for i in twins  if i.origin == origen]
            twins_sorted.append(edge_next[0])
            origen = edge_next[0].queue
            if len(twins_sorted) == len(twins):
                a = False
        #for i in twins_sorted:
         #   print(i.name)
        # Agregamos el next y prev
        for i in range(len(twins_sorted)):
            #print(twins_sorted[i].name)
            if i == 0:
                print(twins_sorted[0].name)
                self.list_edge.append(twins_sorted[0])
            else:
                prev_twin = self.list_edge[-1]
                twins_sorted[i].add_prev(prev_twin)
                if i == len(twins_sorted)-1:
                    index = len(self.list_edge) - len(twins_sorted)
                    prev_twin.add_next(self.list_edge[index])
                else:
                    prev_twin.add_next(self.list_edge[i])
                self.list_edge.append(twins_sorted[i])
                    

    #agregar los seguientes y previos a cada objeto de las semi aristas
    def _add_next(self):
        for i in range(len(self.list_edge)):
            if i == len(self.list_edge)-1:
                self.list_edge[i].add_next(self.list_edge[0])
            else:
                self.list_edge[i].add_next(self.list_edge[i+1])   
    
    def _add_prev(self):
        for i in range(len(self.list_edge)):
            if i == 0:
                self.list_edge[i].add_prev(self.list_edge[len(self.list_edge)-1])
            else:
                self.list_edge[i].add_prev(self.list_edge[i-1])
    
    
    
    #Generaremos la lista de aristas 
    def create_half_edge(self):
        print('Entro')
        for i in range(len(self.list_vertex)): 
            name_edge = 'e'+str(i+1)
            if i == (len(self.list_vertex)-1):
                e = half_edge(self.list_vertex[i], self.list_vertex[0], name_edge)
                self.list_vertex[i].add_edge_inci(e)
            else: 
                e = half_edge(self.list_vertex[i], self.list_vertex[i+1], name_edge)
                self.list_vertex[i].add_edge_inci(e)

            self.list_edge.append(e)

        self._add_next()
        self._add_prev()
        self._add_twin()
    
    #El mas arriba a la izq
    def p_up(self):
        p = self.list_vertex[0]
        for i in range(len(self.list_vertex)):
            if self.list_vertex[i] == p:
                p = p.r_orden(self.list_vertex[i+1])
            else:
                p = p.r_orden(self.list_vertex[i])
        return p
    
    #generar un ciclo 
    def cicle(self,e,f):
        while(e.next != e):
            e.add_face(f)
            e.name = e.name + f.name
            e = e.next

    #Generar lista de Caras
    def create_face(self):
        Inter,exter = face('I'),face('E')
        
        #Punto más arriba a la izquierda 
        p = self.p_up()
        # Asegurarse de tomar la arista e que tenga ese punto como origen y que no sea la gemela
        for i in self.list_edge:
            if (p == i.oringe) and (i.name == 'e'):
                idex = self.list_edge.index(i)
        
        e_inc = self.list_edge[idex]
        start_t,end_t = twin(e_inc)
        e_twin = None
        #Encontrar el gemelo
        for i in self.list_edge:
            if (i.origin == start_t) and (i.queue == end_t):
                e_twin = i

        a = orientacion(e_inc.origin,e_inc.next.origin,e_inc.next.queue)
        if a == True:
            Inter.interior = e_inc
            self.cicle(e_inc,Inter)
            exter.exterior = e_twin
            self.cicle(e_twin,exter)
            print('Las aristas que conformar dicho ciclo son frontera de cara interior')
        else:
            exter.exterior = e_inc
            self.cicle(e_inc,exter)
            Inter.interior = e_twin
            self.cicle(e_twin,exter)
            print('Las aristas que conformar dicho ciclo son frontera de cara exterior')
            
        self.list_face.append(Inter)
        self.list_face.append(exter)





        




    

## Generar Vertice 

In [7]:
def general_vertex(P):
    list_vertex = []
    for i in range(len(P)):
        name = "V"+str(i+1)
        v = vertex(P[i][0],P[i][1], name)
        list_vertex.append(v)
    return list_vertex

## Prueba

In [54]:
P  = [[0,1], [-1,0],[0,-1],[1,0]]
lv = general_vertex(P) #genera los vertex en objetos
LADE = DCEL(lv) #lista de aristas doblemente enlazadas
LADE.create_edge_face()


for i in range(len(LADE.list_edge)):
    print(LADE.list_edge[i].name)
    #print(LADE.list_edge[i].next.name)
    #print(LADE.list_edge[i].prev.name)
 #   print(LADE.list_edge[i].name)

Entro
Entro2
te4
e1
e2
e3
e4
te4
te3
te2
te1
